In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
print(pd.__version__)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2.2.2
/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [3]:
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df.dtypes

id                 int64
model_a           object
model_b           object
prompt            object
response_a        object
response_b        object
winner_model_a     int64
winner_model_b     int64
winner_tie         int64
dtype: object

In [ ]:
from langdetect import detect
from collections import Counter

# Function to detect language in a given text
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'  # handle cases where language detection fails

# Extract prompts and responses
prompts = df['prompt'].tolist()
responses_a= df['response_a'].tolist()
responses_b = df['response_b'].to_list()
tot = responses_a + responses_b
print(prompts)
print(tot)
# Detect languages in prompts and responses
prompt_languages = [detect_language(prompt) for prompt in prompts]
response_languages = [detect_language(tot) for response in responses]
respo
# Combine all detected languages
all_languages = prompt_languages + response_languages

# Count the number of unique languages
unique_languages = Counter(all_languages)

print(f"Number of unique languages detected: {len(unique_languages)}")
print("Languages detected:", unique_languages.keys())


In [4]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=97b635e2d8dd7396e1736e2e3e2974aa82aa0b4fb4cf3c6dc5a475f7985d5113
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset from CSV
# df = pd.read_csv('your_dataset.csv')

# Separate inputs (prompts and responses) and labels (winners)
prompts = df['prompt'].tolist()
responses_a = df['response_a'].tolist()
responses_b = df['response_b'].tolist()
labels_a = df['winner_model_a'].tolist()
labels_b = df['winner_model_b'].tolist()

# Combine responses and labels for model_a and model_b
responses = [(resp_a, resp_b) for resp_a, resp_b in zip(responses_a, responses_b)]
labels = [(label_a, label_b) for label_a, label_b in zip(labels_a, labels_b)]

print("------------------")


------------------


In [4]:
!pip install transformers

In [18]:
import logging

def disable_logging_during_tests():
    # Store the current log level to restore it later
    original_log_level = logging.getLogger().getEffectiveLevel()

    # Set the log level to a higher level, e.g., WARNING or CRITICAL
    logging.disable(logging.CRITICAL)

    # Run your tests here

    # Restore the original log level after the tests
    logging.disable(original_log_level)

# Call this function before running your tests
disable_logging_during_tests()


In [19]:
from transformers import BertTokenizer

# Tokenizer initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',device_map='auto')
tokenizer

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [20]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:

# Tokenize prompts and responses
tokenized_inputs = []
for prompt, (resp_a, resp_b) in zip(prompts, responses):
    inputs_a = tokenizer(prompt, resp_a, padding='max_length', truncation=True, max_length=512, return_tensors='pt').to(device)
    inputs_b = tokenizer(prompt, resp_b, padding='max_length', truncation=True, max_length=512, return_tensors='pt').to(device)
#     print(inputs_a)
    tokenized_inputs.append((inputs_a, inputs_b))

# Prepare input tensors
input_ids = torch.cat([torch.cat([item[0]['input_ids'], item[1]['input_ids']], dim=0) for item in tokenized_inputs], dim=0).to(device)
attention_masks = torch.cat([torch.cat([item[0]['attention_mask'], item[1]['attention_mask']], dim=0) for item in tokenized_inputs], dim=0).to(device)
labels_tensor = torch.tensor(labels)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Split data into train and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels_tensor, random_state=42, test_size=0.2)

# Create TensorDataset and DataLoader for training
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Create TensorDataset and DataLoader for validation
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
from transformers import BertForSequenceClassification, AdamW
import torch.nn as nn
import torch.optim as optim

# Model initialization
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Training loop
num_epochs = 3
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}')

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for batch in val_dataloader:
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
            outputs = model(**inputs)
            loss = outputs.loss
            val_loss += loss.item()
            
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)
            val_correct += torch.sum(preds == batch[2]).item()
    
    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_correct / len(val_dataset)
    print(f'Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}')

# Save the model
torch.save(model.state_dict(), 'bert_model.pth')
